# Packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
from tensorflow.keras.models import load_model

# Firebase
import firebase_admin
from firebase_admin import db
import json

# Turn On GPU

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU') # Return all the GPUS in my device
for gpu in gpus: # For each GPU you found
    tf.config.experimental.set_memory_growth(gpu, True) # Maximize its usage

# Load Data from Google Drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
file_path = '/content/gdrive/MyDrive/Data.csv'

# Load the data using pandas
data = pd.read_csv(file_path)
data = data.dropna()

In [ ]:
model_path = '/content/gdrive/MyDrive/weather_model.h5'
model = load_model(model_path)

In [ ]:
creds_path = '/content/gdrive/MyDrive/creds.json'
cred_obj = firebase_admin.credentials.Certificate(creds_path)
default_app = firebase_admin.initialize_app(cred_obj, {
	'databaseURL':'https://elderly-care-2d133-default-rtdb.firebaseio.com/'
	})

***Model Creation Start***

# Training and Validation Data

In [ ]:
# Split the data into input (X) and output (y) variables
X = data[['Temperature (°C)', 'Humidity (%)']].values
y = data[['Wind Speed (m/s)', 'Maximum Wind Speed (m/s)']].values

In [ ]:
# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating Model

In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(2,)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(2)
])

In [ ]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse') # Mean Squared Error loss for regression

# Model Epochs

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
2084/2084 [==============================] - 17s 5ms/step - loss: 4.3794 - val_loss: 3.9291
Epoch 2/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.6316 - val_loss: 3.2820
Epoch 3/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.4373 - val_loss: 3.2115
Epoch 4/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.3908 - val_loss: 3.1757
Epoch 5/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.3675 - val_loss: 3.2147
Epoch 6/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.3629 - val_loss: 3.2314
Epoch 7/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.3532 - val_loss: 3.3075
Epoch 8/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.3460 - val_loss: 3.3046
Epoch 9/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.3439 - val_loss: 3.2701
Epoch 10/50
2084/2084 [==============================] - 10s 5ms/step - loss: 

In [ ]:
# Train the model with learning rate scheduling
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[lr_scheduler])

Epoch 1/50
2084/2084 [==============================] - 11s 5ms/step - loss: 3.2928 - val_loss: 3.1645 - lr: 0.0010
Epoch 2/50
2084/2084 [==============================] - 8s 4ms/step - loss: 3.3025 - val_loss: 3.1994 - lr: 0.0010
Epoch 3/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.2939 - val_loss: 3.1968 - lr: 0.0010
Epoch 4/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.2913 - val_loss: 3.1216 - lr: 0.0010
Epoch 5/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.3003 - val_loss: 3.0967 - lr: 0.0010
Epoch 6/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.2927 - val_loss: 3.1018 - lr: 0.0010
Epoch 7/50
2084/2084 [==============================] - 10s 5ms/step - loss: 3.2876 - val_loss: 3.1658 - lr: 0.0010
Epoch 8/50
2084/2084 [==============================] - 9s 4ms/step - loss: 3.2938 - val_loss: 3.2777 - lr: 0.0010
Epoch 9/50
2084/2084 [==============================] - 9s 4ms/step - loss:

# Saving Model

In [ ]:
model.save('weather_model.h5')

In [ ]:
# Evaluate the model on the validation set
val_loss = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")

521/521 [==============================] - 1s 2ms/step - loss: 3.1854
Validation Loss: 3.185415029525757


In [ ]:
# Calculate accuracy (you can define a custom metric based on your requirements)
# In regression tasks like this, accuracy is not typically used, but you can calculate metrics like RMSE or MAE.
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 1.784773145381751


In [ ]:
# Predict on validation data
y_pred = model.predict([[23,60]])
print(y_pred)

1/1 [==============================] - 0s 81ms/step
[[2.1773152 4.7605786]]


Model Creation End

# Firebase Code

In [ ]:
def getTempHum():
  ref = db.reference("/ESP/lj496azHzjSQi0hP3GJ65MfFMbA2/current/")
  data = ref.get() #return dictionary with all your data inside that path
  tempHum = []
  temp = 0.0
  hum = 0.0
  for key, value in data.items():
    if(key == "tempSensor"):
      temp = float(value)
    if(key == "humSensor"):
      hum = float(value)
  tempHum.append(temp)
  tempHum.append(hum)

  return tempHum

In [ ]:
def updateWindData():
  ref = db.reference("/PYTHON/lj496azHzjSQi0hP3GJ65MfFMbA2/")
  best_sellers = ref.get() #return dictionary with all your data inside that path
  print(best_sellers.items()) #dict_items, array of tuples, each tuple contain (key"real key of data", value)
  print(type(best_sellers.items()))
  tempHum = [getTempHum()]
  predection = model.predict(tempHum)[0]
  windSpeed = predection[0]
  maxSpeed = predection[1]
  for key, value in best_sellers.items():
    if(key == "windValue"):
      ref.update({"windValue":str(windSpeed)})
    if(key == "maxValue"):
      ref.update({"maxValue":str(maxSpeed)})
  print("Predection Done Successfully")
  ref.update({"predict":"no"})

In [ ]:
while(1):
  ref = db.reference("/PYTHON/lj496azHzjSQi0hP3GJ65MfFMbA2/")
  data = ref.get() #return dictionary with all your data inside that path

  for key, value in data.items():
    if(key == "predict" and value == "yes"):
      updateWindData()
    elif(key == "predict" and value == "no"):
      pass

dict_items([('maxValue', '4.998986'), ('predict', 'yes'), ('windValue', '2.2336907')])
<class 'dict_items'>
1/1 [==============================] - 3s 3s/step
Predection Done Successfully
dict_items([('maxValue', '4.7738214'), ('predict', 'yes'), ('windValue', '2.1716568')])
<class 'dict_items'>
1/1 [==============================] - 0s 31ms/step
Predection Done Successfully
dict_items([('maxValue', '4.7857475'), ('predict', 'yes'), ('windValue', '2.1776443')])
<class 'dict_items'>
1/1 [==============================] - 0s 48ms/step
Predection Done Successfully
dict_items([('maxValue', '4.7857475'), ('predict', 'yes'), ('windValue', '2.1776443')])
<class 'dict_items'>
1/1 [==============================] - 0s 52ms/step
Predection Done Successfully
dict_items([('maxValue', '4.7857475'), ('predict', 'yes'), ('windValue', '2.1776443')])
<class 'dict_items'>
1/1 [==============================] - 0s 56ms/step
Predection Done Successfully


KeyboardInterrupt: ignored